# All Sites- Create Duration Events
* calculating duration of unloading 

In [1]:
# import packages 
%matplotlib widget

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots # adding for subplots
import plotly.figure_factory as ff

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import csv 
import copy 
import os.path 

# Import All Data

## Sodankyla Data

In [2]:
# OPEN MET DATA 
sod =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Sodankyla/SOD_1819_1hr_cleaned.csv")
sod['datetime'] = pd.to_datetime(sod['datetime'])

# OPEN CLASSIFICATIONS
sodobs = pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/datetimeformat_classifications_sodankyla2019_type.csv")
sodobs['datetime'] = pd.to_datetime(sodobs['datetime'])
sodobs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed 

# MERGE
soddf = pd.merge(sod, sodobs, how='outer', on='datetime')
soddf.index = pd.DatetimeIndex(soddf['datetime'])
# soddf.drop(columns=['datetime'], inplace=True)

# Create seperate df for only snow in the canopy timesteps 
sod_df_unload = soddf.copy(deep=True)
sod_df_unload = sod_df_unload.dropna(axis=0, subset=['CD'])

# Have to remove all nans for this plotting to work.... need to come back to this 
# sod_df_unload.dropna(inplace=True)

# Create sunlit column
sod_df_unload['Sunlit'] = sod_df_unload.E.copy(deep=True)
sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
sod_df_unload['Classification'] = sod_df_unload.CD.copy(deep=True)
sod_df_unload.Classification.mask(sod_df_unload.Classification == 0, "Snow Unloading", inplace=True)
sod_df_unload.Classification.mask(sod_df_unload.Classification == 1, "Snow Staying", inplace=True)

sod_df_snowstay   = sod_df_unload.where(sod_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
sod_df_snowunload = sod_df_unload.where(sod_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

In [3]:
#---------------------------------------------------------------------------------------------------------------------------


# https://plotly.com/python/marker-style
# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
sod_df_unload['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
sod_df_unload['Typesymbol'].mask(sod_df_unload['Type'] == 1, 102, inplace=True) 
sod_df_unload['Typesymbol'].mask(sod_df_unload['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_unload['Typesymbol'] = sod_df_unload['Typesymbol'].fillna(128)  

# # SNOW-------------------------------------------------------
# Create symbols for sunlit or not
sod_df_unload['Snowsymbol'] = np.nan

# open triangle for melt, diamond for sub, open x circle for no classification
sod_df_unload['Snowsymbol'].mask(sod_df_unload['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
sod_df_unload['Snowsymbol'].mask(sod_df_unload['Snow'] == 'sub', 18, inplace=True)  
sod_df_unload['Snowsymbol'].mask(sod_df_unload['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_unload['Snowsymbol'] = sod_df_unload['Snowsymbol'].fillna(128)  


#---------------------------------------------------------------------------------------------------------------------------

## Laret

In [4]:
# Open met dataset
opn  = pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/MetStations/Cleaned/opnc_hcass.csv")
opn['datetime'] = pd.to_datetime(opn['datetime'])

# Open observations classifications 
obs = pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/Excel/Datetime/datetimeformat_ccg_classifications_only_cutdates_type.csv")
obs['datetime'] = pd.to_datetime(obs['datetime'])
obs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed

# Merge them
opndf = pd.merge(opn, obs, how='outer', on='datetime')
opndf.index = pd.DatetimeIndex(opndf['datetime'])

# Create seperate df for only snow in the canopy timesteps 
opn_df_unload = opndf.copy(deep=True)
opn_df_unload = opn_df_unload.dropna(axis=0, subset=['CD'])

# # Have to remove all nans for this plotting to work.... need to come back to this 
# opn_df_unload.dropna(inplace=True)

# SPECIAL FOR LARET BECAUSE OF NAMES 
opn_df_unload['temp'] = opn_df_unload['AirTC_Avg'].copy(deep=True)
opn_df_unload['shortwave'] = opn_df_unload['SWR_Avg'].copy(deep=True)
opn_df_unload['windspeed'] = opn_df_unload['WS_ms_Avg'].copy(deep=True)

# Create sunlit column
opn_df_unload['Sunlit'] = opn_df_unload.E.copy(deep=True)
opn_df_unload.Sunlit.mask(opn_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
opn_df_unload.Sunlit.mask(opn_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
opn_df_unload['Classification'] = opn_df_unload.CD.copy(deep=True)
opn_df_unload.Classification.mask(opn_df_unload.Classification == 0, "Snow Unloading", inplace=True)
opn_df_unload.Classification.mask(opn_df_unload.Classification == 1, "Snow Staying", inplace=True)

opn_df_snowstay   = opn_df_unload.where(opn_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
opn_df_snowunload = opn_df_unload.where(opn_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

In [5]:
#---------------------------------------------------------------------------------------------------------------------------

# https://plotly.com/python/marker-style
# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
opn_df_unload['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
opn_df_unload['Typesymbol'].mask(opn_df_unload['Type'] == 1, 102, inplace=True) 
opn_df_unload['Typesymbol'].mask(opn_df_unload['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
opn_df_unload['Typesymbol'] = opn_df_unload['Typesymbol'].fillna(128)  

# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
opn_df_unload['Snowsymbol'] = np.nan

# open triangle for melt, diamond for sub, open x circle for no classification
opn_df_unload['Snowsymbol'].mask(opn_df_unload['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
opn_df_unload['Snowsymbol'].mask(opn_df_unload['Snow'] == 'sub', 18, inplace=True)  
opn_df_unload['Snowsymbol'].mask(opn_df_unload['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
opn_df_unload['Snowsymbol'] = opn_df_unload['Snowsymbol'].fillna(128)  


#---------------------------------------------------------------------------------------------------------------------------

## Niwot

In [6]:
# AMERIFLUX MET DATA 
niwotflux =   pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/niwot_2017_ameriflux_unload1hr.csv")
niwotflux['datetime']  = pd.DatetimeIndex(niwotflux['datetime'])

# OBSERVATIONS CLASSIFICATIONS
niwotobs =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/datetimeformat_classifications_niwot2017_type.csv")
niwotobs['datetime'] = pd.to_datetime(niwotobs['datetime'])

niwotobs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed 

# MERG THEM 
niwotdf = pd.merge(niwotflux, niwotobs, how='outer', on='datetime')
niwotdf.index = pd.DatetimeIndex(niwotdf['datetime'])

# Create seperate df for only snow in the canopy timesteps 
niwot_df_unload = niwotdf.copy(deep=True)
niwot_df_unload = niwot_df_unload.dropna(axis=0, subset=['CD'])

# # Have to remove all nans for this plotting to work.... need to come back to this 
# niwot_df_unload.dropna(inplace=True)

# Create sunlit column
niwot_df_unload['Sunlit'] = niwot_df_unload.E.copy(deep=True)
niwot_df_unload.Sunlit.mask(niwot_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
niwot_df_unload.Sunlit.mask(niwot_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
niwot_df_unload['Classification'] = niwot_df_unload.CD.copy(deep=True)
niwot_df_unload.Classification.mask(niwot_df_unload.Classification == 0, "Snow Unloading", inplace=True)
niwot_df_unload.Classification.mask(niwot_df_unload.Classification == 1, "Snow Staying", inplace=True)

niwot_df_snowstay   = niwot_df_unload.where(niwot_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
niwot_df_snowunload = niwot_df_unload.where(niwot_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

In [7]:
#---------------------------------------------------------------------------------------------------------------------------


# https://plotly.com/python/marker-style
# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
niwot_df_unload['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
niwot_df_unload['Typesymbol'].mask(niwot_df_unload['Type'] == 1, 102, inplace=True) 
niwot_df_unload['Typesymbol'].mask(niwot_df_unload['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwot_df_unload['Typesymbol'] = niwot_df_unload['Typesymbol'].fillna(128)  

# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
niwot_df_unload['Snowsymbol'] = np.nan

# open triangle for melt, diamond for sub, open x circle for no classification
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
niwot_df_unload['Snowsymbol'].mask(niwot_df_unload['Snow'] == 'sub', 18, inplace=True)  
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwot_df_unload['Snowsymbol'] = niwot_df_unload['Snowsymbol'].fillna(128)  


#---------------------------------------------------------------------------------------------------------------------------

# Calculate Duration of Unloading

In [8]:
# THIS IS ITT OMG MAGIC 
opn_events = opn_df_unload.groupby((opn_df_unload["Classification"] != opn_df_unload["Classification"].shift()).cumsum()).agg({"datetime" : ["min", "max", "count"], "shortwave" : ["mean"], "temp" : ["mean"], "windspeed" : ["mean"], "CD" : ["mean"], "Type" : ["mean"]})
sod_events = sod_df_unload.groupby((sod_df_unload["Classification"] != sod_df_unload["Classification"].shift()).cumsum()).agg({"datetime" : ["min", "max", "count"], "shortwave" : ["mean"], "temp" : ["mean"], "windspeed" : ["mean"], "CD" : ["mean"], "Type" : ["mean"]})
niw_events = niwot_df_unload.groupby((niwot_df_unload["Classification"] != niwot_df_unload["Classification"].shift()).cumsum()).agg({"datetime" : ["min", "max", "count"], "shortwave" : ["mean"], "temp" : ["mean"], "windspeed" : ["mean"], "CD" : ["mean"], "Type" : ["mean"]})

In [9]:
opn_events.to_csv('/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/laret_open_events.csv')
sod_events.to_csv('/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/sodankyla_events.csv')
niw_events.to_csv('/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/niwot_events.csv')

In [10]:
# # Only save the unloading events information
# opn_events.CD = opn_events.CD.where(opn_events.CD == 0).dropna()
# opn_events.dropna(inplace=True)

# sod_events.CD = sod_events.CD.where(sod_events.CD == 0).dropna()
# sod_events.dropna(inplace=True)

# niw_events.CD = niw_events.CD.where(niw_events.CD == 0).dropna()
# niw_events.dropna(inplace=True)

# editing the rest manually and then working with them somewhere else, it was too hard to get the formatting correct here